In [1]:
# dobest.io ( 블로그 ) 

# 크롤링 => 스크래핑(링크), 크롤링(페이지), 파싱(추출)
#                     크롤링 = (스크래핑) + 크롤링 + 파싱(*)

In [2]:
# 데이터(리소스)

# 데이터베이스 소스 X
# 윈도우 프로그램 / 웹 사이트 ( 데스크탑, 모바일 ) / 모바일 어플리케이션

In [3]:
# 1. 웹사이트 ( HTML(구조), CSS(스타일), Javascript(동적인 기능) )

# 1.1. 정적인 사이트 ( Server Rendering )
# 1.2. 동적인 사이트 ( Client/Javascript Rendering )
# 1.3. iFrame 으로 이루어진 사이트 ( 네이버 카페, 인트라넷, 공공기관 )
# 1.4. Javascript 으로만 동작하는 사이트

In [4]:
# 1. 크롤링
#      HTML 파일을 다운로드 ( .html )
# 2. 파싱 
#      어떻게 추출해야지 가장 쉽게 할 수 있을까?
#      html, css, js
#      css를 가지고 html element 를 명시하는 방법
#      css selector ( div > ul > li, div#realrank > li )

In [5]:
# https://dobest.io/
url = "https://dobest.io/"

In [6]:
import requests

In [7]:
response = requests.get(url)

In [8]:
# 우리가 찾으려고 하는 것
# 제목들만 리스트 형태로 뽑자.

In [9]:
# article ( class = "post" "." )
# header
# h1
# text

# class ("."), id ("#")
css_selector = "article.post header h1"

In [10]:
# 파싱을 도와주는 라이브러리 - beautifulsoup, lxml

In [11]:
from bs4 import BeautifulSoup

In [12]:
dom = BeautifulSoup(response.text, "html.parser")
# dom ( document object model )
# html > head, body > div, article .. 

In [13]:
title_elements = dom.select(css_selector)

In [14]:
title_element = title_elements[0]

# $ pip install requests ( = web request )
# $ pip install beautifulsoup4

# 네이버 메인페이지의 실시간 검색어 10개 

In [15]:
[
    title_element.select_one("a").text
    for title_element
    in title_elements
]

['postgresql 에서 한글 정렬 문제 해결하기',
 '[강의 소개] Node.js 로 시작하는 웹 프로그래밍',
 '[강의 소개] 업무 자동화를 위한 파이썬',
 'Sentry 를 이용한 Node.js 에러 모니터링',
 '오픈소스 네이밍: 에러 모니터링을 위한 Sentry 와 스타크래프트2',
 'autotrack 을 이용한 구글 어낼리틱스 이벤트 트래킹 자동화',
 'Node.js 에서 한글 인코딩 문제 해결하기']

In [16]:
response = requests.get("http://www.naver.com/")
dom = BeautifulSoup(response.text, "html.parser")

In [17]:
# class => "." => page 내에서 공통점이 있는 elements
# id => "#" => page 내에서 unique 한 element

rank_elements = dom.select("ol#realrank li")

In [18]:
rank_elements = dom.select("ol#realrank li")
rank_element = rank_elements[0]

In [19]:
# rank_element

# 문제제기: 네이버 메인페이지에서 봤던 코드랑 살짝 다르다.
# 이유: 네이버에서 실검 서버 렌더링 되었다가 클라어인트에서 수정됨.

# 방법1: title="최홍만"
# 방법2: a 태그 안에 있는 "최홍만"

In [20]:
# rank_element.select_one("a").text.split("상승")[0]

In [21]:
rank_elements = dom.select("ol#realrank li")
rank_element = rank_elements[0]
rank_element.select_one("a").attrs.get("title")

'어글리몬스터'

In [22]:
[
    rank_element.select_one("a").attrs.get("title")
    for rank_element
    in rank_elements
][:-1]

['어글리몬스터',
 'ufc',
 '박선규',
 '설민석',
 '미세먼지',
 '부산날씨',
 '아버님 제가 모실게요',
 '황우슬혜',
 '김제동',
 'nba']

In [23]:
# 웹 서버 렌더링 
# 1. 크롤링 - pagination, auth, 어떤 알 수 없는 이유?
# 2. 파싱 
# -----------------------------------------------
# 다음 블로그 검색결과 를 크롤링하는 함수 
# input: keyword, page

In [24]:
KEYWORD = "파이썬"
# URL에 표시되지 않을수도 있습니다.
# HTTP Method ( GET, POST )

# http://search.daum.net/search?w=blog&nil_search=btn&DA=NTB&enc=utf8&q=파이썬

# URL Querysting
# http
# search.daum.net
# /search
# data: ?w=blog&q=파이썬

def crawl_daum_blog_page(keyword, page="1"):
    url = "http://search.daum.net/search?w=blog&q={keyword}&page={page}".format(
        keyword=keyword,
        page=str(page),
    )
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    post_elements = dom.select("ul#blogResultUl li")
    print(len(post_elements))
    titles = [
        post_element.select_one(".wrap_tit a").text
        for post_element
        in post_elements
    ]
    return titles

    # 2페이지, 3페이지 ... n 페이지 ( page 정보는 url 에 어떻게 들어갈까? )

crawl_daum_blog_page(KEYWORD, 3)

10


['파이썬학원 찾으시나요?',
 '파이썬 리눅스 함께 공부해야한다!',
 'IT뱅크 파이썬학원 수강료조회',
 '파이썬 프로그래밍 전문교육 학원을 알아보자!',
 '파이썬 설치하기',
 '나노 서버에 파이썬과 장고 실행하기',
 '파이썬 하나면 모든지 가능하다',
 '파이썬(Python) 설치',
 '파이썬 프로그래밍 초보자에게 필요한 1순위 언어',
 '12월 프로그래밍 기초 파이썬 강좌 안내']

In [25]:
# SelectorGadget ( chrome webstore ) - element, class, id
# User-Agent Switcher for Google Chrome

# HTTP Request 
# 1. URL
# 2. HTTP Method ( GET, POST, PUT, PATCH, DELETE )
# 3. Data
# 4. Headers (*) - /search/?q=파이썬
#    - User-Agent: "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2910.0 Safari/537.36"

# HTTP Response

In [26]:
# play.dobest.io ( JAPAN )
response = requests.get("https://search.naver.com/search.naver?where=post&sm=tab_jum&ie=utf8&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC")

In [27]:
# response.text

In [28]:
IPAD_USER_AGENT = "Mozilla/5.0(iPad; U; CPU iPhone OS 3_2 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Version/4.0.4 Mobile/7B314 Safari/531.21.10"

In [29]:
response = requests.get(
    "https://search.naver.com/search.naver?where=post&sm=tab_jum&ie=utf8&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC",
    headers={
        "User-Agent": IPAD_USER_AGENT,
        # Authentication 
    }
)

In [30]:
dom = BeautifulSoup(response.text, "html.parser")

In [31]:
[
    title_element.text
    for title_element
    in dom.select(".total_tit")
]

['파이썬 프로그래밍 초보자에게 필요한 1순위 언어',
 '파이썬학원 기초부터 탄탄하게 배워야합니다',
 '파이썬강좌 멀리서 찾지마세요',
 '파이썬프로그래밍 배워봅시다!',
 '파이썬 프로그래밍 교육 기초부터 알차게 시작하자!',
 '파이썬 공부 쉽게하기',
 '파이썬- 생산성 높은 프로그래밍 언어',
 '[모두의 파이썬] 나도 코딩하는 여자! ^^',
 '파이썬 웹 프로그래밍 강좌 11월소식',
 '파이썬(python) 공부하기 04',
 '안랩쌤과 함께하는 맘잡고 코딩 파이썬 기초과정',
 '파이썬 플라스크 HTML 연결 및 변수 받아오기',
 '파이썬강좌 코드잇에서 공부하세요!',
 '마인크래프트로 배우는 파이썬 프로그래밍 리뷰',
 'Docker로 파이썬 배포 운영하기']